In [3]:
import os

QDRANT_KEY=os.getenv('QDRANT_KEY')
OPENAI_KEY=os.getenv('OPENAI_KEY')
CLUSTER_URL=os.getenv('CLUSTER_URL')

In [4]:
from qdrant_client import QdrantClient

# connect to the cluster
qdrant_client = QdrantClient(
    url=CLUSTER_URL,
    api_key=QDRANT_KEY
)

In [7]:
from qdrant_client.models import Distance, VectorParams

# create the collection
qdrant_client.recreate_collection(
    collection_name="my_collection",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

print("Create collection response:", qdrant_client)

collection_info = qdrant_client.get_collection(collection_name="my_collection")
print("Collection info:", collection_info)

Create collection response: <qdrant_client.qdrant_client.QdrantClient object at 0x122a06d90>
Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=0 indexed_vectors_count=0 points_count=0 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) payload_schema={}


In [39]:
import pdfplumber

pages_to_skip = [2, 40, 41]
one_pagers = [1, 3, 4, 7, 10, 14, 18, 21, 24, 28, 34, 38, 43, 44]
CROP_SIZE = 0.9

fulltext = ""
with pdfplumber.open("../Data/JDRFTEENTOOLKIT.pdf") as pdf:
    for page in pdf.pages:
        if page.page_number in pages_to_skip:
            continue
        if page.page_number in one_pagers:
            page = page.crop((0, 0, float(page.width), CROP_SIZE * float(page.height)))
            fulltext += page.extract_text()
        else:
            left = page.crop((0, 0, 0.5 * float(page.width), CROP_SIZE * float(page.height)))
            right = page.crop((0.5 * float(page.width), 0, page.width, CROP_SIZE * float(page.height)))
        
            left_text = left.extract_text()
            right_text = right.extract_text()
        
            fulltext += left_text
            fulltext += '\n'
            fulltext += right_text
            
        fulltext += '\n'
        
print(fulltext)

Teen Toolkit
Navigating the teenage years with type 1 diabetes can be difficult. This guide
can help parents and teens make the journey a much more pleasant—and even
enjoyable—experience.
Introduction
For many parents, the teen years loom ahead like some giant shadow. Even
the most confident, involved family worries about how they will maneuver
the teen years.
JDRF understands, and hopes with this toolkit, parents will find a starting
point and what to expect and how to deal with the many issues facing teens
with type 1 diabetes. From driving to drinking to hormones to newer school
issues, we try to help you think ahead, plan ahead and be ready.
But remember, this is a toolkit meant to be used with other programs. Reach
out to your JDRF chapter and find out about meet ups for teens with diabe-
tes and for parents of teens and for advocacy programs to make all of you
feel like you are making a difference.
The teen years do pass, and with some special care and work, we make
it through. T

In [41]:
# divide in chunks

text = fulltext
chunks = []
chunk_length = 500

while text:
    next_text = text[:chunk_length]
    last_period_index = next_text.rfind('.')
    if last_period_index == -1:
        last_period_index = chunk_length
    text = text[last_period_index + 1:]

    next_chunk = next_text[:last_period_index + 1]
    chunks.append(next_chunk)
    
for chunk in chunks:
    print(chunk)
    print('---')

Teen Toolkit
Navigating the teenage years with type 1 diabetes can be difficult. This guide
can help parents and teens make the journey a much more pleasant—and even
enjoyable—experience.
Introduction
For many parents, the teen years loom ahead like some giant shadow. Even
the most confident, involved family worries about how they will maneuver
the teen years.
---

JDRF understands, and hopes with this toolkit, parents will find a starting
point and what to expect and how to deal with the many issues facing teens
with type 1 diabetes. From driving to drinking to hormones to newer school
issues, we try to help you think ahead, plan ahead and be ready.
But remember, this is a toolkit meant to be used with other programs.
---
 Reach
out to your JDRF chapter and find out about meet ups for teens with diabe-
tes and for parents of teens and for advocacy programs to make all of you
feel like you are making a difference.
The teen years do pass, and with some special care and work, we make
it 

In [52]:
len(chunks)

259

In [58]:
# create the embeddings for the entire text
# To test, I only take the first 30 chunks because openai has limits
from openai.error import RateLimitError
import time

from qdrant_client.http.models import PointStruct
import openai

openai.api_key = OPENAI_KEY

points = []
i = 1
for chunk in chunks:
    i += 1
    print(i)
    # print("Embeddings chunk:", chunk)
    while True:
        try:
            response = openai.Embedding.create(
                input=chunk,
                model="text-embedding-ada-002"
            )
            break
        except RateLimitError as e:
            print('sleeping...')
            time.sleep(70)
            print('sleep over')
    
    embeddings = response['data'][0]['embedding']
    
    points.append(PointStruct(id=i, vector=embeddings, payload={"text": chunk}))

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
sleeping...
sleep over
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
sleeping...
sleep over
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
sleeping...
sleep over
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260


In [59]:
len(points)

259

In [61]:
# add the embeddings to the collection

operation_info = qdrant_client.upsert(
    collection_name="my_collection",
    wait=True,
    points=points
)

print("Operation inf:", operation_info)

Operation inf: operation_id=2 status=<UpdateStatus.COMPLETED: 'completed'>


In [65]:
def create_answer_with_context(query, messages):
    # create embedding for query
    response = openai.Embedding.create(
        input=query,
        model="text-embedding-ada-002"
    )
    
    embeddings = response['data'][0]['embedding']
    
    # search for similar embeddings
    search_result = qdrant_client.search(
        collection_name="my_collection",
        query_vector=embeddings,
        limit=5
    )
    
    prompt = "Context:\n"
    
    for result in search_result:
        prompt += result.payload["text"] + "\n---\n"
    prompt += "Question:" + query + "\n---\n" + "Answer:"
    # print(prompt)
    # add prompt to openai gpt-3.5
    
    messages.append(
        {"role": "user", "content": prompt}
    )
    
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    answer = completion.choices[0].message.content
    
    messages.append(
        {"role": "assistant", "content": answer}
    )
    
    return answer, messages

In [68]:
messages = []
input = "Are hemoglobin A1c levels of teens higer or lower compared to adults on average?"
answer, messages = create_answer_with_context(input, messages)

In [69]:
answer

'Hemoglobin A1c levels of teens are one percent higher on average compared to adults.'

In [70]:
input = "How much higher is it?"
answer, messages = create_answer_with_context(input, messages)

In [71]:
answer

'Teens with T1D have an average hemoglobin A1c level that is one percent higher than that of adults.'

In [72]:
messages

[{'role': 'user',
  'content': 'Context:\n\nSo what exactly does puberty do to T1D overall? According to\nmedical studies, teens have an average hemoglobin A1c level\nthat is one percent higher than that of adultsiii. As a result,\nthey typically require more insulin. To complicate matters,\nthe sex hormones (estrogen and testosterone) that are the\nhallmark of puberty work against insulin. While insulin lowers\nblood glucose, sex hormones raise it.\n---\n A teen has school,\nsports, other activities, a social life, and more. It could be that\nyour teen is away from you more than he is home. So how is\na parent supposed to recognize if any rebellion is going on?\nOften, an elevated HbA1c level can be the first sign that makes\nparents wonder if their teen is starting to rebel. But an elevated\nHbA1c is not reason to immediately scream, “rebellion!”\nsince hormones and growth can affect HbA1c levels.\n---\n Large swings\nin HbA1c levels are one red flag. (By that we mean that\nyour medi

In [73]:
input = "What is the result of that?"
answer, messages = create_answer_with_context(input, messages)

In [74]:
answer

"Teens may begin to question their parents' requests regarding managing their blood sugar, or even ignore and avoid them. They may also become more self-conscious about their diabetes management tools, such as pumps and meters, and seek to hide them."

In [75]:
input = "What does this say about the insuline intake?"
answer, messages = create_answer_with_context(input, messages)

In [76]:
answer

'Teens with T1D typically require more insulin during puberty, as their average hemoglobin A1c level is one percent higher than that of adults. However, the sex hormones (estrogen and testosterone) that are the hallmark of puberty work against insulin, so it can be a challenge for parents to keep up with which increases in insulin dosage are due to more food and which are due to changes in the growing teen’s body. Taking the amount of insulin needed to keep blood glucose levels in a safe and healthy range does not lead to weight gain. Additionally, cutting back on insulin to lose weight quickly can lead to unhealthy eating behavior.'